In [1]:
import pandas as pd
import geopandas as gpd
from shapely import geometry


In [2]:
#departamentos AGBA segun INDEC
departamentos = {
    'Comuna 1':'02001',
    'Comuna 2':'02002',
    'Comuna 3':'02003',
    'Comuna 4':'02004',
    'Comuna 5':'02005',
    'Comuna 6':'02006',
    'Comuna 7':'02007',
    'Comuna 8':'02008',
    'Comuna 9':'02009',
    'Comuna 10':'02010',
    'Comuna 11':'02011',
    'Comuna 12':'02012',
    'Comuna 13':'02013',
    'Comuna 14':'02014',
    'Comuna 15':'02015',
    'ALMIRANTE BROWN':'06028',
    'AVELLANEDA':'06035',
    'BERAZATEGUI':'06091',
    'ESCOBAR':'06252',
    'ESTEBAN ECHEVERRIA':'06260',
    'EZEIZA':'06270',
    'FLORENCIO VARELA':'06274',
    'GENERAL RODRIGUEZ':'06364',
    'GENERAL SAN MARTIN':'06371',
    'HURLINGHAM':'06408',
    'ITUZAINGO':'06410',
    'JOSE C PAZ':'06412',
    'LA MATANZA':'06427',
    'LANUS':'06434',
    'LOMAS DE ZAMORA':'06490',
    'MALVINAS ARGENTINAS':'06515',
    'MARCOS PAZ':'06525',
    'MERLO':'06539',
    'MORENO':'06560',
    'MORON':'06568',
    'PILAR':'06638',
    'PRESIDENTE PERON':'06648',
    'QUILMES':'06658',
    'SAN FERNANDO':'06749',
    'SAN ISIDRO':'06756',
    'SAN MIGUEL':'06760',
    'SAN VICENTE':'06778',
    'TIGRE':'06805',
    'TRES DE FEBRERO':'06840',
    'VICENTE LOPEZ':'06861'
}

departamentosName = {
    '02001':'Comuna 1',
    '02002':'Comuna 2',
    '02003':'Comuna 3',
    '02004':'Comuna 4',
    '02005':'Comuna 5',
    '02006':'Comuna 6',
    '02007':'Comuna 7',
    '02008':'Comuna 8',
    '02009':'Comuna 9',
    '02010':'Comuna 10',
    '02011':'Comuna 11',
    '02012':'Comuna 12',
    '02013':'Comuna 13',
    '02014':'Comuna 14',
    '02015':'Comuna 15',
    '06028':'ALMIRANTE BROWN',
    '06035':'AVELLANEDA',
    '06091':'BERAZATEGUI',
    '06252':'ESCOBAR',
    '06260':'ESTEBAN ECHEVERRIA',
    '06270':'EZEIZA',
    '06274':'FLORENCIO VARELA',
    '06364':'GENERAL RODRIGUEZ',
    '06371':'GENERAL SAN MARTIN',
    '06408':'HURLINGHAM',
    '06410':'ITUZAINGO',
    '06412':'JOSE C PAZ',
    '06427':'LA MATANZA',
    '06434':'LANUS',
    '06490':'LOMAS DE ZAMORA',
    '06515':'MALVINAS ARGENTINAS',
    '06525':'MARCOS PAZ',
    '06539':'MERLO',
    '06560':'MORENO',
    '06568':'MORON',
    '06638':'PILAR',
    '06648':'PRESIDENTE PERON',
    '06658':'QUILMES',
    '06749':'SAN FERNANDO',
    '06756':'SAN ISIDRO',
    '06760':'SAN MIGUEL',
    '06778':'SAN VICENTE',
    '06805':'TIGRE',
    '06840':'TRES DE FEBRERO',
    '06861':'VICENTE LOPEZ'
}



In [3]:
#CARGO CARTOGRAFIA NACIONAL NO OFICIAL
pais = gpd.GeoDataFrame.from_file('../dt/carto/paisNoOficial/rad_paisp.shp')
#CREO ID REDCODE PARA TODAS LAS PROVINCIAS PORQUE CABA NO TIENE LINK
pais['REDCODE'] = pais.PROV + pais.DEPTO + pais.FRAC + pais.RADIO

#CREO EL CODIGO DEPARTAMENTO PARA FILTRAR LOS PERTENECIENTES A PARTIDOS DEL AGBA
pais['DEPARTAMENTO'] = pais.REDCODE.map(lambda x: x[0:5])

#ELIMINAMOS LOS DEPARTAMENTOS QUE NO SEAN DE AGBA SEGUN INDEC
pais = pais.loc[pais.DEPARTAMENTO.isin(departamentos.values()),:]

#LOS RADIOS FALTANTES DE CABA SON URBANOS
pais.TIPO.fillna('U',inplace=True)

#ELIMINAMOS LOS RADIOS RURALES
pais = pais.loc[pais.TIPO!='R',:]

#AGREGAMOS ETIQUETAS A LOS DEPARTAMENTOS
deptos = []
for i in range(pais.shape[0]):
    deptos.append(departamentosName[pais.DEPARTAMENTO.iloc[i]])
pais['DEPTO_NAME'] = deptos
pais = pais.to_crs(epsg=22185)
pais.head()

,AREA,DEPTO,FRAC,LINK,PERIMETER,PROV,RADIO,RADPAIS_,RADPAIS_ID,TIPO,geometry,REDCODE,DEPARTAMENTO,DEPTO_NAME
26688,16948780.0,252,01,062520101,26902.490,06,01,26690,26690,M,"POLYGON ((4158793.5 6185928, 4158824.25 618588...",062520101,06252,ESCOBAR
26704,12351870.0,252,01,062520102,30469.020,06,02,26706,26707,M,"POLYGON ((4159777 6185123.5, 4159847 6185160, ...",062520102,06252,ESCOBAR
26719,11063170.0,252,02,062520208,25350.040,06,08,26721,26723,U,"POLYGON ((4161392.5 6184536.5, 4161399.75 6184...",062520208,06252,ESCOBAR
26741,796720.3,252,02,062520201,6843.383,06,01,26743,26743,U,"POLYGON ((4164165.5 6183091, 4164223.5 6183021...",062520201,06252,ESCOBAR
26747,2996825.0,252,02,062520202,8393.760,06,02,26749,26749,U,"POLYGON ((4166774.25 6182343.5, 4166960.5 6182...",062520202,06252,ESCOBAR


In [7]:
pais.to_file('../dt/carto/agba')

## load density

In [61]:
    #probar aca http://catalogo.datos.gba.gob.ar/dataviews/230038/radios-censales/
pba = gpd.read_file('../dt/carto/Codgeo_Buenos_Aires_con_datos/Buenos_Aires_con_datos.shp')
pba = pba.loc[:,['link','totalpobl']]
pba.columns = ['REDCODE','pob']
pba.head()

,REDCODE,pob
0,068821704,2003.0
1,060141201,56.0
2,060210206,177.0
3,060070107,216.0
4,060210609,80.0


In [62]:
caba = gpd.read_file('../dt/carto/Codgeo_CABA_con_datos/cabaxrdatos.shp')
caba = caba.loc[:,['LINK','TOT_POB']]
caba.columns = ['REDCODE','pob']
caba.head()

,REDCODE,pob
0,020130302,654.0
1,020131101,568.0
2,020130104,763.0
3,020130103,612.0
4,020130102,770.0


In [63]:
caba.loc[~caba.REDCODE.isin(pais.REDCODE),:]

,REDCODE,pob
2202,020061201,522.0
2881,020071205,740.0


In [64]:
paraMerge = caba.app
end(pba)
pais = pais.merge(paraMerge, how='left')
pais['densidad'] = pais.pob / pais.AREA
pais['densidad'] = pd.qcut(pais.densidad,10,labels=False)

In [74]:
pais['densidad'] = pd.qcut(pais.densidad,10,labels=False)

In [76]:
paisBajaDensidad = pais.loc[pais.densidad==0,:]
paisBajaDensidad.head()
paisBajaDensidad.to_file('../dt/carto/agbaDensiBaja')

In [ ]:
#SACAR DECILES DE DENCIDAD Y QUEDARME CON EL 1 DECIL

pais.to_file('../dt/carto/agba')

## Load OSM

In [31]:
#https://mapzen.com/data/metro-extracts/metro/buenos-aires_argentina/
osm = gpd.read_file('../dt/carto/buenos-aires_argentina.osm2pgsql-shapefiles/buenos-aires_argentina_osm_line.shp')
osm = osm.loc[:,['highway','geometry']]
osm.head()

,highway,geometry
0,unclassified,"LINESTRING (-59.2536487 -35.0956878, -59.25235..."
1,unclassified,"LINESTRING (-59.2460124 -35.0346348, -59.24599..."
2,unclassified,"LINESTRING (-59.2556364 -35.027778, -59.255110..."
3,None,"LINESTRING (-59.256935 -35.0258877, -59.246128..."
4,unclassified,"LINESTRING (-59.2547303 -35.0182057, -59.25593..."


In [33]:
mask =  (osm.highway == 'residential') | (osm.highway == 'unclassified') |  (osm.highway == 'living_street') 

In [34]:
osm2 = osm.loc[mask,:]
osm2 = osm2.to_crs(epsg=22185)
osm2.to_file('../dt/carto/osmResidential')

In [79]:
paisBajaDensidad.head()

,AREA,DEPTO,FRAC,LINK,PERIMETER,PROV,RADIO,RADPAIS_,RADPAIS_ID,TIPO,geometry,REDCODE,DEPARTAMENTO,DEPTO_NAME,pob,densidad
0,16948780.0,252,01,062520101,26902.490,06,01,26690,26690,M,"POLYGON ((5605573.523703439 6208535.44525929, ...",062520101,06252,ESCOBAR,597.0,0.0
1,12351870.0,252,01,062520102,30469.020,06,02,26706,26707,M,"POLYGON ((5606597.563514996 6207794.426813126,...",062520102,06252,ESCOBAR,1009.0,0.0
2,11063170.0,252,02,062520208,25350.040,06,08,26721,26723,U,"POLYGON ((5608236.390336967 6207306.631047663,...",062520208,06252,ESCOBAR,208.0,0.0
3,796720.3,252,02,062520201,6843.383,06,01,26743,26743,U,"POLYGON ((5611075.149383005 6206034.538516479,...",062520201,06252,ESCOBAR,281.0,0.0
4,2996825.0,252,02,062520202,8393.760,06,02,26749,26749,U,"POLYGON ((5613709.611741914 6205445.958880841,...",062520202,06252,ESCOBAR,426.0,0.0


In [90]:
#http://catalogo.datos.gba.gob.ar/dataviews/232662/manzanas/
manzanas = pd.read_csv('/home/pipe/Downloads/manzanas.csv',usecols=[2])
manzanas.head()

,coordenadas
0,"-63.23964037201355,-37.45716462906551,0.0 -63...."
1,"-63.23976119742675,-37.45725926272377,0.0 -63...."
2,"-63.23981305454304,-37.460301595881454,0.0 -63..."
3,"-63.241806494307596,-37.46015847807085,0.0 -63..."
4,"-63.25172350409261,-37.45086428064242,0.0 -63...."


In [158]:
#Y es 30s
#X es 60s
poligonList=[]
for i in range(manzanas.shape[0]):
    pointList = []
    texto = manzanas.coordenadas[i].replace('0.0','').replace(' ','')
    lista = str.split(texto,',')

    if len(lista)%2>0:
        del lista[-1]

    for i in range(len(lista)):
        if i % 2== 0:
            #i es x
            #Point(float(row['lon']), float(row['lat']))

            pointList.append(geometry.Point(float(lista[i]),float(lista[i+1])))         
        else:
            pass
    coords = [(p.x, p.y) for p in pointList]
    poly = Polygon(coords)
    poligonList.append(poly)


crs = {'init': 'epsg:4326'}
geo_df = gpd.GeoDataFrame(manzanas, crs=crs, geometry=poligonList)

In [162]:
geo_df = geo_df.to_crs(epsg=22185)
geo_df.to_file('../dt/carto/manzanas')

In [ ]:
#clip las calles solo dentro de los poligonos de baja densidad
#para cada poligono, conseguir alguna metrica de densidad de calles polygonizr
#